In [54]:
import requests
from bs4 import BeautifulSoup as bs 
import json
import pandas as pd 
import numpy as np
import lxml
import time
import pickle

In [3]:
df = pd.read_csv("C:/Users/UICHEOL_HWANG\Edu_file/my_python_lab/망고플레이트.csv")

In [68]:
df["ID"].loc[df["주소"].notnull()&df["주소"].str.contains("광화문")]

6      DOqnqbIeugJI
19     dzvLL3cnX1UC
20     g-ZF2ky7D3m-
207      wT1_KWIfeZ
228    vkE5tx5MgycG
277      GMd0yzrobn
336    JjS1bklwYEM6
375    qKcTCTPwY_VP
394    KV9lSGa6xNQp
477    HN1U16KgCI24
528    t6oxGDjInBYA
627      JlEOAtsbIt
666    -czYKuW77F_5
702    EQB1ieEn8jac
729      KxinsNEdHO
747    iVAHP06TG-ai
749      5w3XQVn5Ce
779      myslZBeMRB
802      9vHLBTjW3m
Name: ID, dtype: object

In [62]:
df[df["ID"] =="lNGf0ef7vT"]

,Unnamed: 0,링크,상점명,주소,카테고리,조회수,리뷰,ID
12980,12980,/restaurants/lNGf0ef7vT,비비비당,부산 해운대구,카페 / 디저트,"311,533",100,lNGf0ef7vT


In [55]:
from tqdm import tqdm
keyword = ["종로구","중랑구","성북구","강북구","도봉구","강동구","송파구","영등포구","관악구","강남구","서초구","금천구","노원구","동작구",
          "구로구","금천구","마포구","양천구","광진구","용산구","고양시","의정부시","광명시","성남시","오산시","부천시","안산시"]


data_list = []

for word in tqdm(keyword): # 키워드로 한 번 돌려주면서 
    for i in range(1, 51): # 키워드 내 50페이지까지 크롤링
        url = f"https://www.mangoplate.com/search/{word}?keyword={word}&page={i}"
        header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
        # 자신이 봇이 아니란것을 확인시켜주기 위한 인증내역 (망고플레이트는 필요 )
        try:
            r = requests.get(url, headers=header) #요청 받기 
            r.raise_for_status() #요청 성공내역 확인 
            time.sleep(5) #5초동안 쉬었다가 시작
        
            soup = bs(r.text, "lxml") # lxml로 파싱한 내용 변환
            for restaurant_item in soup.select(".restaurant-item"):
                restaurant_info = {} #식당 정보들을 반복적으로 담아줄 빈 딕셔너리 
                image = restaurant_item.select_one(".thumb >img")["data-original"] # 이미지 링크 
                link = restaurant_item.select_one(".info a")['href'] # 페이지 이동 링크 
                name = restaurant_item.select_one(".title").text.strip().replace("\n", "").replace(" ", "") # 식당이름 
                address = restaurant_item.select_one(".etc").text.split("-")[0].strip() # 주소
                category = restaurant_item.select_one(".etc").text.split("-")[1].strip() # 카테고리 
                view_count = restaurant_item.select_one(".view_count").text # 조회수 
                review_count = restaurant_item.select_one(".review_count").text # 리뷰 수 

                restaurant_info["링크"] = link
                restaurant_info["이미지"] = image
                restaurant_info["상점명"] = name
                restaurant_info["주소"] = address
                restaurant_info["카테고리"] = category
                restaurant_info["조회수"] = view_count
                restaurant_info["리뷰"] = review_count
                data_list.append(restaurant_info)
                
        except Exception as e:
            print(f"키워드 '{word}'의 {i}번째 페이지에서 오류 발생: {e}")



100%|███████████████████████████████████████████████████████████████████████████████| 27/27 [2:13:08<00:00, 295.88s/it]


In [57]:
data = pd.DataFrame(data_list)

In [59]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26287 entries, 0 to 26286
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   링크      26287 non-null  object
 1   이미지     26287 non-null  object
 2   상점명     26287 non-null  object
 3   주소      26287 non-null  object
 4   카테고리    26287 non-null  object
 5   조회수     26287 non-null  object
 6   리뷰      26287 non-null  object
dtypes: object(7)
memory usage: 1.4+ MB


In [63]:
data.to_csv("망고플레이트_수도권.csv",encoding="utf-8",index=False)

In [64]:
data

,링크,이미지,상점명,주소,카테고리,조회수,리뷰
0,/restaurants/l0k43OGTH50J,https://mp-seoul-image-production-s3.mangoplat...,데시데,삼청/인사,카페 / 디저트,"14,766",28
1,/restaurants/6g1d24chargz,https://mp-seoul-image-production-s3.mangoplat...,호라파,종로,태국 음식,"13,459",17
2,/restaurants/cOPpYpvBoWrH,https://mp-seoul-image-production-s3.mangoplat...,우육면관(광화문점),종로,기타 중식,"64,938",50
3,/restaurants/CxTm9IzfC5-x,https://mp-seoul-image-production-s3.mangoplat...,효제루,종로,정통 중식 / 일반 중식,"12,821",25
4,/restaurants/dzvLL3cnX1UC,https://mp-seoul-image-production-s3.mangoplat...,모던샤브하우스(광화문D타워점),광화문,탕 / 찌개 / 전골,"97,735",34
...,...,...,...,...,...,...,...
26282,/restaurants/GIi8er3gLkce,https://mp-seoul-image-production-s3.mangoplat...,미리내우리밀칼국수,안산시,국수 / 면 요리,144,1
26283,/restaurants/oOzK2rikn7rH,https://mp-seoul-image-production-s3.mangoplat...,알프스돈까스,안산시,기타 한식,"4,752",6
26284,/restaurants/kZIGcdPA1vwd,https://mp-seoul-image-production-s3.mangoplat...,오마카세,안산시,회 / 스시,"4,526",5
26285,/restaurants/L5yeuDloYrhq,https://mp-seoul-image-production-s3.mangoplat...,젊은이의초밥,안산시,회 / 스시,"4,061",5
